### CNN 
이번장에서는 이미지 인식 분야에서 매우 좋은 성능으 보이는 Convolution Neural Network을 이용하여 MNIST데이터를 학습시켜 보고 앞서 배운 기본적인 신경망보다 성능이 얼마나 좋아지는지 확인해 봅시다.

In [3]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot = True)

X = tf.placeholder(tf.float32,[None,28,28,1])
Y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


당연히 시작은 tensorflow와 mnist를 임포트 해줍니다.   

다음으로 X의 첫번째 차원인 ```None```은 입력데이터의 개수입니다. 그리고 앞장에서 28x28=784짜리 하나의 차원으로 구성했던것과 달리 2차원 평면으로써 ```28,28```로써 더 직관적으로 표현해줍니다. 마지막으로 ```1```은 MNIST데이터가 회색조 이미지라 채널에 색상이 한개 뿐이라 1을 사용합니다.   
Y는 0~9중 한가지로 분류하므로 10으로 만들어 주며 Dropout을 위한 keep_prob 플레이스홀더도 정의해줍니다.

In [0]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1], padding= 'SAME')
L1 = tf.nn.relu(L1)
init = tf.global_variables_initializer()


이제 첫 번째 CNN계층을 구성해 봅니다.   
이미지를 convolution시킬 필터의 크기를 3x3x1 2차원으로 지정하고 32개의 필터를 만들어 냅니다.   
다음으로 ```tf.nn.conv2d```를 이용해 ```strides```와 ```padding```을 정해주며 convolution을 시켜줍니다. 

In [0]:
L1= tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

다음으로 ```tf.nn.max_pool```함수를 이용해 최대값풀링 계층을 만들어줍니다.
여기서 커널의 크기(```ksize```)는 2x2로 하는 풀링계층을 만들고 ```strides=[1,2,2,1]```값은 슬라이딩 시 두칸씩 움직임을 뜻합니다.

In [0]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1], padding= 'SAME')
L2 = tf.nn.relu(L2)
L2= tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

위와 같이 2번째 CNN계층을 만들어 줍니다.   
다만 이번에는 W2가 ```[3,3,32,64]```로 3x3x32(1번째 계층의 커널수) 크기의 필터를 64개 만들어 줍니다.   
여기서 2번째 계층의 출력은 32x64개의 층이 아닌 64층입니다.


In [0]:
W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev=0.01))
L3 = tf.reshape(L2,[-1,7*7*64])
L3 = tf.matmul(L3,W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

이제 추출한 특징들을 이용해 10개의 분류를 만들어 내는 계층을 구성해 봅니다.   
먼저 10개의 분류는 1차원 배열이므로 차원을 줄이는 단계를 거쳐야 합니다. 직전의 풀링 계층의 크기가 7x7x64이므로 ```tf.reshape``` 함수를 이용해 7x7x64=3136의 1차원 계층으로 만듭니다.   
이 배열을 W3과 곱해 256개의 뉴런으로 연결하는 신경망을 만들어줍니다.   
추가로 Dropout기법 역시 사용해줍니다.

In [0]:
W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
model = tf.matmul(L3,W4)

이제 모델 구성의 마지막으로 L3의 256개의 출력값을 받아 0~9 레이블을 갖는 10개의 출력값을 만듭니다.

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

늘 하던식으로 최적화함수를 만들어 줍니다.

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
  total_cost = 0
  
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape(-1,28,28,1)

    _, cost_val = sess.run([optimizer, cost], feed_dict = {X:batch_xs, Y: batch_ys, keep_prob:0.7})
    total_cost += cost_val
  
  print('Epoch:', '%04d' % (epoch+1),'Avg. cost =', '{:.3f}' .format(total_cost/total_batch))
print('최적화 완료')

Epoch: 0001 Avg. cost = 0.331
Epoch: 0002 Avg. cost = 0.099
Epoch: 0003 Avg. cost = 0.073
Epoch: 0004 Avg. cost = 0.056
Epoch: 0005 Avg. cost = 0.046
Epoch: 0006 Avg. cost = 0.042
Epoch: 0007 Avg. cost = 0.033
Epoch: 0008 Avg. cost = 0.028
Epoch: 0009 Avg. cost = 0.027
Epoch: 0010 Avg. cost = 0.022
Epoch: 0011 Avg. cost = 0.020
Epoch: 0012 Avg. cost = 0.017
Epoch: 0013 Avg. cost = 0.016
Epoch: 0014 Avg. cost = 0.016
Epoch: 0015 Avg. cost = 0.013
최적화 완료


In [0]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('정확도:',sess.run(accuracy, feed_dict = {X: mnist.test.images.reshape(-1,28,28,1), Y: mnist.test.labels, keep_prob : 1}))

정확도: 0.9894
